# Question 2

## Simplex
定义一个叫<code>SimplexMethod</code>的函数，它使用标准格式(默认$x\succeq 0$)的线性规划问题的$c,A,b,x0$作为输入，输出迭代最后的结果，元组类型:$(solver,point)$<br>
计算逻辑如下：
1. 首先使用<code>Find_x0</code>计算初值.
2. 然后使用<code>SimplexMehthod</code>进行单纯形法求最优值:
<ul>
    <li>在单纯形法函数中定义了几个辅助函数，其中一个<code>compute_X</code>是计算的主要过程,考虑到单纯形法过程可以用递归的形式实现，在<code>compute_X</code>中完成了递归调用自己来计算的过程</li>
    <li>其他辅助函数基本上都是简单的计算一些必要矩阵、或者进行标准化等零碎工作的，没有必要细看</li>
</ul>

In [1]:
import numpy as np
import pandas as pd
def Find_x0(c,A,b):
    """Take the standard form of c ,A ,b as input,can return a x0 as the initial point"""
    B=np.array(A[:,0],dtype="float").reshape(A.shape[0],1)
    rA=np.linalg.matrix_rank(A)
    rB=1
    if rA<A.shape[0]:
        print("警告：矩阵A不满秩")
    for i in range(0,A.shape[1]):
        if np.linalg.matrix_rank(np.append(B,A[:,i].reshape(A.shape[0],1).reshape(A.shape[0],1),axis=1))==rA:
            B=np.append(B,A[:,i].reshape(A.shape[0],1),axis=1)
            x=np.linalg.solve(B,b)
            return np.append(x,np.zeros(A.shape[1]-rA))
        elif np.linalg.matrix_rank(np.append(B,A[:,i].reshape(A.shape[0],1),axis=1))>rB:
            B=np.append(B,A[:,i].reshape(A.shape[0],1),axis=1)
    x=np.linalg.solve(B,b)
    return np.append(x,np.zeros(A.shape[1]-len(x)))
def SimplexMethod(c,A,b,x0):
    """The implement of Simplex Method, t
    he input concluse an objective vector c , the paramter A and b and intial point x0"""
    
    #辅助函数区
    ##单纯形计算的主函数
    def compute_X(B,inv_B,N,x):
        """迭代进行单纯形法计算：
        这一个程序的功能是给出inv_B,N,可以计算出下一步走的方向，并更新相对应的值"""
        #遍历N的所有列     
        for q in range(0,N.shape[1]):
            #计算dq，此处应该顺便把dq旋转一下,使其和x的顺序是一致的
            dq=np.append(-np.dot(inv_B,N[:,q]),e(q))
            dq=dq_rotate(A,B,N,dq)
            #如果发现dq是一个使函数下降的方向，则判断lam的取值
            if is_cdle0(dq):
                lam=np.infty
                for j in range(0,dq.size):
                    if dq[j]<0:
                        lam=min(lam,-x[j]/dq[j])
                x=x+np.dot(lam,dq)
                #lam是infty,说明函数无界
                if lam==np.infty:
                    return (-np.infty,x)
                B_new,N=compute_B_N(A,x)
                #遍历B的所有列，如果哪一列和B_new不同，就提取出来作为转变列
                for i in range(0,B_new.shape[1]):
                    if np.any(B_new[:,i]!=B[:,i]):
                        u=B_new[:,i]-B[:,i]
                        v=e(i,length=B.shape[1])
                        break
                # 用SMWF算法计算invB'
                inv_B=inv_B-np.dot(np.outer(np.dot(inv_B,u),v),inv_B)/(1+np.dot(np.dot(v,inv_B),u))
                B=B_new
                return compute_X(B,inv_B,N,x)
        #如果没有发现这样的dq
        return (np.dot(c,x),x)
    ## 辅助计算B，N的函数
    def compute_B_N(A,x):
        """给我A,x，可以计算B和N"""
        B=np.array([])
        N=np.array([])
        for i in range(0,x.size):
            if x[i]==0:
                N=np.append(N,A[:,i])
            else:
                B=np.append(B,A[:,i].reshape(A.shape[0],1))
        B=B.reshape(A.shape[0],A.shape[0]).T
        N=N.reshape(A.shape[1]-A.shape[0],A.shape[0]).T
        return B,N
    ## 辅助标准化dq的函数
    def dq_rotate(A,B,N,dq):
        """这个函数可以通过列变换，把dq的次序变成[x1,x2,x3,x4,...]"""
        #先复制一波，以免出现什么幺蛾子
        A_copy=A.copy()
        B_copy=B.copy()
        N_copy=N.copy()
        B_N=np.append(B_copy,N_copy,axis=1)
        #通过遍历A的所有列，对于A的每一列，找到【B，N】里面的相等的那一列，同时对dq进行相同的操作
        for i in range(0,A_copy.shape[1]):
            for j in range(i,B_N.shape[1]):
                if np.all(A_copy[:,i]==B_N[:,j]):
                    temp=B_N[:,j]
                    B_N[:,j]=B_N[:,i]
                    B_N[:,i]=temp
                    dq[i],dq[j]=dq[j],dq[i]
                    break
        return dq
    ## 辅助给出eq的函数
    def e(q,length=A.shape[1]-A.shape[0]):
        """给出一个数q,求得eq"""
        eq=np.array([0]*length)
        eq[q]=1
        return eq

    def is_cdle0(dq):
        """检查是否c.T*dq<0"""
        return np.sum(c.T*dq)<0

    
    # 标准化输入，全部标准化为ndarray
    c=np.array(c)
    A=np.array(A)
    b=np.array(b)
    x=np.array(x0)
    B,N=compute_B_N(A,x)
    #先初始化计算inv_B
    inv_B=np.linalg.inv(B)
    #在调用comput_X函数进行迭代计算
    return compute_X(B,inv_B,N,x)
    


In [2]:
## 定义函数
c=np.array([-5,-1,0,0])
A=np.array([[1,1,1,0],[2,1/2,0,1]])
b=np.array([5,8])

In [3]:
#求一个初值x0
x0=Find_x0(A=A,c=c,b=b)

#使用单纯形法计算
res=SimplexMethod(c=c,A=A,b=b,x0=x0)
print("最优值：",res[0])
print("点x：",res[1])

最优值： -20.0
点x： [4. 0. 1. 0.]


##  Interior Point Method
1. 首先用<code>FindInitialPoint</code>函数求最优值一个初值.
2. 之后调用<code>InteriorPointMethod</code>内点法主函数来进行计算，使用<code>do...while</code>的形式完成主要的梯度下降过程
<ul><li>主函数会调用<code>Fu,Ja_Fu</code>分别计算$F_{u},\nabla F_{u}$</li>
    <li>也会调用<code>LineSearch</code>线搜索合适的步长</li></ul>

In [4]:
def FindInitialPoint(A,b,c):
    """给我一个标准问题，可以找到初始点"""
    #利用A的伪逆计算x0
    x0=np.dot(np.linalg.pinv(A),b)
    #化成diag矩阵的形式
    X0=np.diag(x0)
    #令初始u0=1
    u0=1
    #利用Xλ1=u1计算λ
    lambda0=np.dot(np.linalg.pinv(X0),np.ones_like(x0))
    # v0乱取
    v0=np.ones(A.shape[0])
    return (x0,lambda0,v0,u0)

def InteriorPointMethod(A,b,c,x0,lambda0,v0,u0,epsilon):
    """输入A,b,c,返回内点法计算的结果，结果是tuple格式:(最优值，x)"""
    ##初始化
    t=0
    lam=lambda0
    x=x0
    v=v0
    u=u0
    n=len(x)
    F_u=Fu(u,x,lam,v,A,b,c)
    Ja_Fu=JaFu(A,lam,x)
    point=np.append(x,lam)
    point=np.append(point,v)
    #主要迭代过程
    while np.linalg.norm(F_u)>=epsilon:
        Δpoint=-np.dot(np.linalg.inv(Ja_Fu),F_u)
        point,x,lam,v=LineSearch(point,Δpoint,u,x,lam,v,A,b,c)#定义LineSearch方法可以自动更新point,x,lam,v
        u=0.1/n*np.dot(x,lam)
        #更新F_u，Ja_Fu
        F_u=Fu(u,x,lam,v,A,b,c)
        Ja_Fu=JaFu(A,lam,x)
        t=t+1
    return (np.dot(c,x),x)
def Fu(u,x,lam,v,A,b,c):
    """返回F_u的值"""
    X=np.diag(x)
    Lam=np.diag(lam)
    one=np.ones_like(x)
    Fu=np.append(np.dot(A.T,v)+c-lam,np.dot(A,x)-b)
    Fu=np.append(Fu,np.dot(X,Lam).dot(one)-np.dot(u,one))
    return Fu
def JaFu(A,lam,x):
    """返回Fu的Jacobi矩阵"""
    X=np.diag(x)
    Lam=np.diag(lam)    
    m,n=A.shape
    Zero_n=np.zeros([n,n])
    Zero_m=np.zeros([m,m])
    Zero_mn=np.zeros([m,n])
    Zero_nm=np.zeros([n,m])
    I=np.eye(n)
    Ja_Fu1=np.append(Zero_n,-1*I,axis=1)
    Ja_Fu1=np.append(Ja_Fu1,A.T,axis=1)
    Ja_Fu2=np.append(A,Zero_mn,axis=1)
    Ja_Fu2=np.append(Ja_Fu2,Zero_m,axis=1)
    Ja_Fu3=np.append(Lam,X,axis=1)
    Ja_Fu3=np.append(Ja_Fu3,Zero_nm,axis=1)
    Ja_Fu=np.append(Ja_Fu1,Ja_Fu2,axis=0)
    Ja_Fu=np.append(Ja_Fu,Ja_Fu3,axis=0)
    return Ja_Fu

def LineSearch(point,Δpoint,u,x,lam,v,A,b,c):
    """LineSearch方法，可以返回合适的point,x,lam,v"""
    α=20
    n=len(x)
    m=len(v)
    Δx=Δpoint[0:n]
    Δlam=Δpoint[n:n+n]
    Δv=Δpoint[n+n:]
    if np.any(lam+α*Δlam<0):
        is_fit=0
    elif np.any(x+α*Δx<0):
        is_fit=0
    elif np.linalg.norm(Fu(u,x+α*Δx,lam+α*Δlam,v+α*Δv,A,b,c),ord=None)>\
        0.99*np.linalg.norm(Fu(u,x,lam,v,A,b,c),ord=None):
        is_fit=0
    else:
        is_fit=1
    while is_fit!=1:
        α=1/2*α
        if np.any(lam+α*Δlam<0):
            is_fit=0
        elif np.any(x+α*Δx<0):
            is_fit=0
        elif np.linalg.norm(Fu(u,x+α*Δx,lam+α*Δlam,v+α*Δv,A,b,c),ord=None)>\
        0.99*np.linalg.norm(Fu(u,x,lam,v,A,b,c),ord=None):
            is_fit=0
        else:
            is_fit=1
    x=x+α*Δx
    lam=lam+α*Δlam
    v=v+α*Δv
    point=np.append(x,lam)
    point=np.append(point,v)    
    return point,x,lam,v

In [5]:
## 定义函数
c=np.array([-5,-1,0,0])
A=np.array([[1,1,1,0],[2,1/2,0,1]])
b=np.array([5,8])

In [6]:
## 用内点法计算
# 先求最优值函数初值
x0,lambda0,v0,u0=FindInitialPoint(A,b,c)

#利用内点法计算
optval,x=InteriorPointMethod(A,b,c,x0,lambda0,v0,u0,epsilon=1e-11)
print("最优值：",optval)
print("点x:",x)

最优值： -19.999999999989644
点x: [4.00000000e+00 2.09335659e-11 1.00000000e+00 2.04908898e-12]


#  Question 4
Interior Point Method :定义的内点法和上面的类似（不同之处主要在于修改了<code>FindInitialPoint</code>的计算逻辑），实现了讲义中的标准问题的求最优值，实践过程中调用了该求最优值器最优值决了作业中的问题

In [7]:
def FindInitialPoint(A,b,C,d):
    """给我一个标准问题，可以找到初始点"""
    #利用A的伪逆计算x0
    m,n=A.shape
    l,n=C.shape
    I_m=np.eye(m)
    I_l=np.eye(l)
    C_I=np.append(C,I_l,axis=1)
    x0_s0=np.dot(np.linalg.pinv(C_I),d)
    x0=x0_s0[:n]
    s0=x0_s0[n:]
    if np.any(s0<0):
        s0=0.5*np.ones_like(s0)
        x0=np.dot(np.linalg.pinv(C),d-s0)
    #化成diag矩阵的形式
    X0=np.diag(x0)
    S0=np.diag(s0)
    #令初始u0=1
    u0=1
    #利用VS1=u1计算λ
    v0=np.dot(np.linalg.pinv(S0),np.ones_like(s0))
    return (x0,s0,v0,u0)

def InteriorPointMethod(A,b,C,d,s0,v0,x0,u0,epsilon):
    """输入A,b,c,返回内点法计算的结果，结果是tuple格式:(最优值，x,s)"""
    ##初始化
    t=0
    x=x0
    s=s0
    v=v0
    u=u0
    m,n=A.shape
    F_u=Fu(u,A,b,C,d,s,v,x)
    Ja_Fu=JaFu(A,b,C,d,s,v,x)
    point=np.append(s,v)
    point=np.append(point,x)
    while np.linalg.norm(F_u)>=epsilon:
       # pdb.set_trace()
        Δpoint=-np.dot(np.linalg.inv(Ja_Fu),F_u)
        point,x,s,v=LineSearch(point,Δpoint,u,A,b,C,d,s,v,x)#定义LineSearch方法可以自动更新point,x,lam,v
        u=0.1/n*np.dot(s,v)
        #更新F_u，Ja_Fu
        F_u=Fu(u,A,b,C,d,s,v,x)
        Ja_Fu=JaFu(A,b,C,d,s,v,x)
        t=t+1
    return (np.linalg.norm(A*x-b)**2-17,x,s)
def Fu(u,A,b,C,d,s,v,x):
    """返回F_u的值"""
    X=np.diag(x)
    V=np.diag(v)
    S=np.diag(s)
    one_m=np.ones_like(s)
    one_n=np.ones_like(x)
    Fu=np.append(np.dot(C,x)+s-d,np.dot(V,S).dot(one_m)-np.dot(u,one_m))
    Fu=np.append(Fu,np.dot(np.dot(A.T,A),x)-np.dot(A.T,b)+np.dot(C.T,v))
    return Fu
def JaFu(A,b,C,d,s,v,x):
    """返回Fu的Jacobi矩阵"""
    X=np.diag(x)
    V=np.diag(v)
    S=np.diag(s)
    m,n=A.shape
    l,n=C.shape
    Zero_n=np.zeros([n,n])
    Zero_l=np.zeros([l,l])
    Zero_ln=np.zeros([l,n])
    Zero_nl=np.zeros([n,l])
    I=np.eye(n)
    I_m=np.eye(m)
    I_l=np.eye(l)
    Ja_Fu1=np.append(I_l,Zero_l,axis=1)
    Ja_Fu1=np.append(Ja_Fu1,C,axis=1)
    Ja_Fu2=np.append(V,S,axis=1)
    Ja_Fu2=np.append(Ja_Fu2,Zero_ln,axis=1)
    Ja_Fu3=np.append(Zero_nl,C.T,axis=1)
    Ja_Fu3=np.append(Ja_Fu3,np.dot(A.T,A),axis=1)
    Ja_Fu=np.append(Ja_Fu1,Ja_Fu2,axis=0)
    Ja_Fu=np.append(Ja_Fu,Ja_Fu3,axis=0)
    return Ja_Fu
def LineSearch(point,Δpoint,u,A,b,C,d,s,v,x):
    """LineSearch方法，可以返回合适的point,x,lam,v"""
    α=20
    n=len(x)
    m=len(v)
    Δs=Δpoint[0:m]
    Δv=Δpoint[m:m+m]
    Δx=Δpoint[m+m:]
    if np.any(s+α*Δs<0):
        is_fit=0
    elif np.any(v+α*Δv<0):
        is_fit=0
    elif np.linalg.norm(Fu(u,A,b,C,d,s+Δs,v+α*Δv,x+α*Δx),ord=None)>\
        0.99*np.linalg.norm(Fu(u,A,b,C,d,s,v,x),ord=None):
        is_fit=0
    else:
        is_fit=1
    while is_fit!=1:
        α=1/2*α
        if np.any(s+α*Δs<0):
            is_fit=0
        elif np.any(v+α*Δv<0):
            is_fit=0
        elif np.linalg.norm(Fu(u,A,b,C,d,s+Δs,v+α*Δv,x+α*Δx),ord=None)>\
            0.99*np.linalg.norm(Fu(u,A,b,C,d,s,v,x),ord=None):
            is_fit=0
        else:
            is_fit=1
    x=x+α*Δx
    s=s+α*Δs
    v=v+α*Δv
    point=point+α*Δpoint    
    return point,x,s,v

In [8]:
#定义问题
A=np.eye(2)
b=np.array([1,4])
C=np.array([[1/2,1],[-1,1],[-1,-1],[0,-1]])
d=np.array([1,2,0,0])

In [9]:
# 求一个初值
x0,s0,v0,u0=FindInitialPoint(A,b,C,d)

#求一个最优值
res=InteriorPointMethod(A,b,C,d,s0,v0,x0,u0,epsilon=1e-5)
print("最优值：",res[0]-17)
print("x：",res[1])
print("s:",res[2])

最优值： -7.19999287434786
x： [-0.39998831  1.19999288]
s: [1.27240443e-06 4.00018809e-01 8.00004573e-01 1.19999288e+00]
